# This program connects to Box.com, gets all of the folders and images and stores information into the database.

https://developer.box.com/en/guides/authentication/oauth2/with-sdk/

In [ ]:
!pip install boxsdk
!pip install auth
!pip install redis
!pip install mysql.connector
!pip install requests

# Run the cell below to authenticate with Box.com so we can use the API

In [15]:
from boxsdk import OAuth2

import json
#Set the file we want to use for authenticating a Box app
#The json file stores the client_id and client_secret so we don't have it in the code.
# The json file looks like this:
#{
#"client_id":"___the_codes_for_client_id___",
#"client_secret":"___the_codes_for_client_secret___"
#}

computer_environment="cu"

if(computer_environment=="cu"):
    settings_path = "/home/student/jeff_not_data_5000/programs/"

if(computer_environment=="home"):
    settings_path = "C:\\ProgramData\\MySQL\\MySQL Server 8.0\\"
    
oauth_settings_file = settings_path+'box_app_test.json'
with open(oauth_settings_file, "r") as read_file:
    oauth_data = json.load(read_file)
print(oauth_data["client_id"])
print(oauth_data["client_secret"])

oauth = OAuth2(
    client_id=oauth_data["client_id"],
    client_secret=oauth_data["client_secret"]
)

auth_url, csrf_token = oauth.get_authorization_url('https://jeffblackadar.ca')
print("Click on this:")
print(auth_url)
print(csrf_token)
print("Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.")


rvy5o5ic72c00ldi8vpcqmof0t7h6w85
VWJQ1LTracitLw5HfG8Q0TKplBogFEIT
Click on this:
https://account.box.com/api/oauth2/authorize?state=box_csrf_token_u4yR1JhKlktVzFvY&response_type=code&client_id=rvy5o5ic72c00ldi8vpcqmof0t7h6w85&redirect_uri=https%3A%2F%2Fjeffblackadar.ca
box_csrf_token_u4yR1JhKlktVzFvY
Copy the code that follows code= in the URL.  Paste it into the oauth.authenticate('___the_code___') below.  Be quick, the code lasts only a few seconds.


In [16]:
from boxsdk import Client

# Make sure that the csrf token you get from the `state` parameter
# in the final redirect URI is the same token you get from the
# get_authorization_url method to protect against CSRF vulnerabilities.
#assert 'THE_CSRF_TOKEN_YOU_GOT' == csrf_token
access_token, refresh_token = oauth.authenticate('d6nb0wP7cJvktAKIaHIhGOOBZyH8cUxL')
client = Client(oauth)

# Skip below steps for normal run


## Test the connection to Box.com - list all folders and files using recursion

In [3]:
def process_subfolder_test(client, folder_id, folder_name):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder_test(client, item.id,folder_name+"/"+item.name)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))            
    return

process_subfolder_test(client, '0',"")

this folder: 
Folder 103443832656 is named "Current_Files"
this folder: /Current_Files
File 617353504182 is named "PPM-10ocr.pdf"
File: 617353504182 is named: "PPM-10ocr.pdf" path: /Current_Files/PPM-10ocr.pdf 
File 617327143278 is named "PPM-11ocr2.pdf"
File: 617327143278 is named: "PPM-11ocr2.pdf" path: /Current_Files/PPM-11ocr2.pdf 
File 616469405347 is named "PPM-2ocr4.pdf"
File: 616469405347 is named: "PPM-2ocr4.pdf" path: /Current_Files/PPM-2ocr4.pdf 
File 617355000157 is named "PPM-3ocr.pdf"
File: 617355000157 is named: "PPM-3ocr.pdf" path: /Current_Files/PPM-3ocr.pdf 
File 617315249258 is named "PPM-4ocr2.pdf"
File: 617315249258 is named: "PPM-4ocr2.pdf" path: /Current_Files/PPM-4ocr2.pdf 
File 616470979901 is named "PPM-5ocr2.pdf"
File: 616470979901 is named: "PPM-5ocr2.pdf" path: /Current_Files/PPM-5ocr2.pdf 
File 617349457888 is named "PPM-6ocr2.pdf"
File: 617349457888 is named: "PPM-6ocr2.pdf" path: /Current_Files/PPM-6ocr2.pdf 
File 617354620409 is named "PPM-7ocr2.pdf"
Fi

File 580391413078 is named "colorschememapping.xml"
File: 580391413078 is named: "colorschememapping.xml" path: /pinp5/Altars/Altars/Altar 10612_files/colorschememapping.xml 
File 580400103148 is named "filelist.xml"
File: 580400103148 is named: "filelist.xml" path: /pinp5/Altars/Altars/Altar 10612_files/filelist.xml 
File 580410547851 is named "image001.jpg"
File: 580410547851 is named: "image001.jpg" path: /pinp5/Altars/Altars/Altar 10612_files/image001.jpg 
File 580410668287 is named "image002.jpg"
File: 580410668287 is named: "image002.jpg" path: /pinp5/Altars/Altars/Altar 10612_files/image002.jpg 
File 580415446275 is named "image003.jpg"
File: 580415446275 is named: "image003.jpg" path: /pinp5/Altars/Altars/Altar 10612_files/image003.jpg 
File 580391976241 is named "image004.jpg"
File: 580391976241 is named: "image004.jpg" path: /pinp5/Altars/Altars/Altar 10612_files/image004.jpg 
File 580407355906 is named "image005.jpg"
File: 580407355906 is named: "image005.jpg" path: /pinp5/A

KeyboardInterrupt: 

## Preparation - Get all folders and store all files into a database.

In [ ]:
def process_subfolder(client, folder_id, folder_name,cursor_insert):
    print("this folder: "+folder_name)
    items = client.folder(folder_id=folder_id).get_items()
    for item in items:
        print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))
        if(item.type.capitalize()=="Folder"):
            process_subfolder(client, item.id,folder_name+"/"+item.name,cursor_insert)
        if(item.type.capitalize()=="File"):
            #print(item)
            print('File: {0} is named: "{1}" path: {2} '.format(item.id, item.name, folder_name+"/"+item.name))
            
            insert_image = ("INSERT INTO tbl_box_images (`id_box_file`,`id_box_folder`,`folder`,`file_name`,`img_src`,`img_url`) VALUES (%s,%s,%s,%s,%s,%s);")
            
            cursor_insert.execute(insert_image, (item.id, folder_id, folder_name, item.name, folder_name+"/"+item.name, ""))
            mariadb_connection.commit()
    return





In [ ]:
import requests
import shutil
from pathlib import Path
base_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\pompeii\\box'


import mysql.connector as mariadb

mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")

cursor_insert = mariadb_connection.cursor(buffered=True)

process_subfolder(client, '0',"", cursor_insert)
    
mariadb_connection.close()    


In [ ]:
mariadb_connection.close()    

In [ ]:
#This is out of order - just putting it here for now
#Clean up the 0
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files='C:\\ProgramData\\MySQL\\MySQL Server 8.0\\webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_inner = mariadb_connection.cursor(buffered=True)
cursor_update = mariadb_connection.cursor(buffered=True)
cursor.execute("SELECT id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src FROM webpage_images.tbl_box_images WHERE image_hash is null= 0")
for id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src in cursor:
    print(id, id_box_file, file_name, img_path_match, id_tbl_webpage_images, img_src)
    #don't include .xml, .htm* or .thmx
    #for row 158348
    if(img_path_match is None):
        img_path_match = ""
    if (img_src.lower().find(".xml")<0) and (img_src.lower().find(".htm")<0) and (img_src.lower().find(".thmx")<0) and (img_src.lower().find(".js")<0) and (img_src.lower().find(".pdf")<0):
        download_process_box_thumbnail_id_box_file(client, id, str(id_box_file), folder, file_name, cursor_update)
        if(id_tbl_webpage_images is not None):
            if(not id_tbl_webpage_images>0):
                #logic repeats below - making a function seemed to make this more complicated
                #No match found in the table tbl_box_images
                wi_id=0            
                cursor_wi_id = mariadb_connection.cursor(buffered=True, dictionary=True)
                img_path_match_sql = img_path_match.replace("'","''")
                #print(img_path_match_sql)
                cursor_wi_id.execute("SELECT id as wi_id FROM webpage_images.tbl_webpage_images WHERE img_path_match='"+img_path_match_sql+"';")
                #print(cursor_inner)
                #print("rows "+str(cursor_wi_id.rowcount))
                for wi_ids in cursor_wi_id:
                    wi_id=wi_ids["wi_id"]
                    print("id "+str(wi_id))
                    if(wi_id>0):
                        update_image = ("UPDATE tbl_box_images SET `id_tbl_webpage_images` = %s WHERE id=%s;")
                        #print(update_image)
                        cursor_update.execute(update_image, (wi_id, id))
                        mariadb_connection.commit()
                cursor_wi_id.close()
                #if(wi_id<1):
                #    download_process_box_thumbnail(client, id, str(id_box_file), folder, file_name, cursor_update)
        else:
            #logic repeats from above
            wi_id=0            
            cursor_wi_id = mariadb_connection.cursor(buffered=True, dictionary=True)
            #cursor_inner.execute(select_match, (img_path_match))
            
            img_path_match_sql = img_path_match.replace("'","''")
            #print(img_path_match_sql)
            cursor_wi_id.execute("SELECT id as wi_id FROM webpage_images.tbl_webpage_images WHERE img_path_match='"+img_path_match_sql+"';")
            #print("rows "+str(cursor_wi_id.rowcount))
            for wi_ids in cursor_wi_id:
                wi_id=wi_ids["wi_id"]
                print("id "+str(wi_id))
                if(wi_id>0):
                    update_image = ("UPDATE tbl_box_images SET `id_tbl_webpage_images` = %s WHERE id=%s;")
                    #print(update_image)
                    cursor_update.execute(update_image, (wi_id, id))
                    mariadb_connection.commit()
            cursor_wi_id.close()
            #if(wi_id<1):
            #    download_process_box_thumbnail(client, id, str(id_box_file), folder, file_name, cursor_update)
            
cursor.close()
cursor_update.close()
cursor_inner.close()
mariadb_connection.close()    



In [ ]:
!pip install --upgrade azure-cognitiveservices-vision-computervision

# Set up Microsoft Azure Connection

In [17]:
import os

import json
#Set the file we wan!pip install --upgrade azure-cognitiveservices-vision-computervisiont to use for authenticating an Azure app
#The json file stores the COMPUTER_VISION_SUBSCRIPTION_KEY and COMPUTER_VISION_ENDPOINT so we don't have it in the code.
# The json file looks like this:
#{
#"COMPUTER_VISION_SUBSCRIPTION_KEY":"___the_COMPUTER_VISION_SUBSCRIPTION_KEY___",
#"COMPUTER_VISION_ENDPOINT":"___the_COMPUTER_VISION_ENDPOINT___"
#}

cv_settings_file = settings_path+'azure_app_cv.json'
with open(cv_settings_file, "r") as read_file:
    cv_auth_data = json.load(read_file)
#print(cv_auth_data["COMPUTER_VISION_SUBSCRIPTION_KEY"])
#print(cv_auth_data["COMPUTER_VISION_ENDPOINT"])

os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']=cv_auth_data["COMPUTER_VISION_SUBSCRIPTION_KEY"]
os.environ['COMPUTER_VISION_ENDPOINT']=cv_auth_data["COMPUTER_VISION_ENDPOINT"]

In [18]:
# https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/quickstarts-sdk/python-sdk
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import TextOperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import TextRecognitionMode
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

In [19]:
# Add your Computer Vision subscription key to your environment variables.
if 'COMPUTER_VISION_SUBSCRIPTION_KEY' in os.environ:
    subscription_key = os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']
else:
    print("\nSet the COMPUTER_VISION_SUBSCRIPTION_KEY environment variable.\n**Restart your shell or IDE for changes to take effect.**")
    sys.exit()
# Add your Computer Vision endpoint to your environment variables.
if 'COMPUTER_VISION_ENDPOINT' in os.environ:
    endpoint = os.environ['COMPUTER_VISION_ENDPOINT']
else:
    print("\nSet the COMPUTER_VISION_ENDPOINT environment variable.\n**Restart your shell or IDE for changes to take effect.**")
    sys.exit()

In [20]:
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

## Process images: Connect to Box.com, download image, get its hash value.

In [21]:
# Image hash functions
# import the necessary packages
# Credit to: Adrian Rosebrock https://www.pyimagesearch.com/
from imutils import paths
import argparse
import time
import sys
import cv2
import os

# hashing with OpenCV and PythonPython
def dhash(image, hashSize=8):
    # resize the input image, adding a single column (width) so we
    # can compute the horizontal gradient
    resized = cv2.resize(image, (hashSize + 1, hashSize))

    # compute the (relative) horizontal gradient between adjacent
    # column pixels
    diff = resized[:, 1:] > resized[:, :-1]

    # convert the difference image to a hash
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])



In [22]:
import os
import shutil
from pathlib import Path
if(computer_environment=="cu"):
    base_dir= "/storage/box/"

if(computer_environment=="home"):
    base_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\pompeii\\box'
    


# using the thumbnail for image hashing does not work   
def download_process_box_full_id_box_file(client, id_box, id_box_file, folder, file_name, cursor_update, image_hash_existing,cursor_aitb,cursor_aitb_x,cursor_update_aitb,cursor_update_aitb_x):
    #Some file names can't be processed.  This uses the box file name instead.
    print("this file: "+id_box_file+" "+file_name)
    #using the the large 320 pixel thumbnail produces different results than the full image - it does not work
    #try:
    #file_info = client.file(id_box_file).get()
    #print("file_name: "+file_name)
    place=file_name.rfind(".")    
    ext = file_name[place:]
    if(computer_environment=="cu"):
        
        print("file_name: "+file_name)
    if(computer_environment=="home"):
        file_name = id_box_file+ext
        print("file_name: "+file_name)
    
    # load the image for the image hash later.

    img_local_path=base_dir+folder+"/"+file_name
    #print(img_local_path)
    img_file_name = file_name
    img_local_folder = base_dir+folder
    print(img_local_folder+'/'+ img_file_name)
    #see if the image is there already
    if not os.path.exists(img_local_path):
        print('IMAGE NOT FOUND')
        img_file_name = file_name
        img_local_folder = base_dir+folder
        
        if not os.path.exists(img_local_folder):
            #os.mkdir(img_local_folder)
            path = Path(img_local_folder)
            path.mkdir(parents=True,exist_ok=True)
            #print(img_local_folder+'/'+ img_file_name)
            
            # Open the url image, set stream to True, this will return the stream content.
            # Open a local file with wb ( write binary ) permission.
            #local_file = open(img_local_folder+'/'+ img_file_name, 'wb')
        imagecontent = client.file(id_box_file).content()
        
        with open(img_local_folder+'/'+ img_file_name,'wb') as f:
                f.write(imagecontent)
            #print(thumbnail)
        del imagecontent
        f.close()
            #image hash
    imageHash = 0
    #print("image hash")
    if not os.path.exists(img_local_path):
        print("Error image not loaded for image hash   " + img_local_path)
    else:         
        # load the image from disk
        path = Path(img_local_folder)
        os.chdir(path)
        hash_image = cv2.imread(img_file_name)
        
        # if the image is None then we could not load it from disk (so skip it)
        if not hash_image is None:
            # convert the image to grayscale and compute the hash
            hash_image = cv2.cvtColor(hash_image, cv2.COLOR_BGR2GRAY)
            imageHash = dhash(hash_image)
            #print(imageHash)
        else:
            imageHash = 0
            #print("image hash - can't find image?")
        update_image = ("UPDATE tbl_box_images SET image_hash = %s WHERE id = %s;")
        if(image_hash_existing==imageHash):
            print("the same ", id_box, imageHash,image_hash_existing)
        else:
            #print(id_box)
            #print(imageHash)
            #print(image_hash_existing)
            if(image_hash_existing is None):
                image_hash_existing=0
            print("different ", id_box, imageHash, image_hash_existing,int(imageHash), int(image_hash_existing) )
            #print(bin(int(image_hash_existing)))
            #print(bin(int(imageHash)))
        cursor_update.execute(update_image, (imageHash, id_box))
        mariadb_connection.commit()
        #Azure tags
        azure_img_tags=""
        azure_img_tags_confidence=""
        local_image = open(img_local_path,"rb")
        file_length_in_bytes = os.path.getsize(img_local_path)
        print("file_length_in_bytes %d" % file_length_in_bytes)
        if(file_length_in_bytes >= 4194304):
            #Thanks to https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
            temp_image = cv2.imread(img_local_path)
            print('Original Dimensions : ',temp_image.shape)
            scale_percent = 1024/temp_image.shape[1] # percent of original size
            width = int(temp_image.shape[1] * scale_percent)
            height = int(temp_image.shape[0] * scale_percent)
            dim = (width, height)
            # resize image
            resized = cv2.resize(temp_image, dim, interpolation = cv2.INTER_AREA)
            print('Resized Dimensions : ',resized.shape)
            #cv2.imwrite(base_dir+"temp"+ext, temp_image)
            #local_image = open(base_dir+"temp"+ext,"rb")
            #cv2.imshow("Resized image", resized)
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()
            azure_img_tags="Image too large > 4mb"
            azure_img_tags_confidence=""

            #import inspect
            #print(inspect.getmembers(local_image))
        else:
            if(hash_image.shape[0]>50 and hash_image.shape[1]>50):
                '''
                Tag an Image - remote
                This example returns a tag (key word) for each thing in the image.
                '''
                try:
                    # Call API with remote image
                    tags_result_remote = computervision_client.tag_image_in_stream(local_image)

                    if (len(tags_result_remote.tags) == 0):
                        azure_img_tags="No tags detected."
                        azure_img_tags_confidence=""
                    else:
                        for tag in tags_result_remote.tags:
                            azure_img_tags=azure_img_tags+"["+tag.name+"] "
                            #We have a tag.  Get its ID and then insert it into the cross reference table tbl_azure_img_tags_x with the ID of the image
                            tag=tag.name
                            tag=tag.replace("'","''")
                            #cursor_aitb,cursor_aitb_x,cursor_update_aitb,cursor_update_aitb_x
                            #look to see if the tag is in the tag table tbl_azure_img_tags_box
                            cursor_aitb.execute("SELECT count(*) as cn FROM webpage_images.tbl_azure_img_tags_box WHERE azure_img_tag_box='"+tag+"'")
                            for cn_aitb in cursor_aitb:
                                #if count = 0, not found so insert it
                                if(cn_aitb[0]==0):
                                    cursor_update_aitb.execute("INSERT INTO tbl_azure_img_tags_box (azure_img_tag_box) VALUES ('"+tag+"')")
                                    mariadb_connection.commit()
                                cursor_aitb.execute("SELECT id as aitb_id FROM webpage_images.tbl_azure_img_tags_box WHERE azure_img_tag_box='"+tag+"'")
                                for aitb_id in cursor_aitb:
                                    #print(ait_id)
                                    cursor_aitb_x.execute("SELECT count(*) as cn FROM webpage_images.tbl_azure_img_tags_box_x WHERE aitb_id=%s AND bi_id=%s",(aitb_id[0],id_box))
                                    for cn_aitb_x in cursor_aitb_x:
                                        #print("count " + str(cn[0]))
                                        #if count = 0, not found so insert it
                                        if(cn_aitb_x[0]==0):
                                            cursor_update_aitb_x.execute("INSERT INTO tbl_azure_img_tags_box_x (aitb_id,bi_id) VALUES (%s, %s)",(aitb_id[0],id_box))
                                            mariadb_connection.commit()
                except Exception as e:
                    print(e)
                    azure_img_tags="Image not found."
                    azure_img_tags_confidence=""
            
            else:
                azure_img_tags="Image too small to tag"
                azure_img_tags_confidence=""
        local_image.close()
        print(azure_img_tags)
            #os.remove(img_local_path)
    return
    #except:
    #    print(sys.exc_info()[0])
    #    return
    

In [ ]:
number_of_rows_to_run = 100000
start_id = 250047
end_id = start_id+number_of_rows_to_run

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(option_files=settings_path+'webpage_images.cnf', option_groups="webpage_images")
cursor = mariadb_connection.cursor(buffered=True)
cursor_inner = mariadb_connection.cursor(buffered=True)
cursor_update = mariadb_connection.cursor(buffered=True)
cursor_aitb = mariadb_connection.cursor(buffered=True)
cursor_aitb_x = mariadb_connection.cursor(buffered=True)
cursor_update_aitb = mariadb_connection.cursor(buffered=True)
cursor_update_aitb_x = mariadb_connection.cursor(buffered=True)

cursor.execute("SELECT id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src, image_hash FROM webpage_images.tbl_box_images WHERE id>=%s AND id<%s", (start_id, end_id))
for id, id_box_file, folder, file_name, img_path_match, id_tbl_webpage_images, img_src, image_hash in cursor:
    #print(id, id_box_file, img_path_match, id_tbl_webpage_images, img_src)
    #don't include .xml, .htm* or .thmx
    #for row 158348
    if(img_path_match is None):
        img_path_match = ""
    if (img_src.lower().find(".xml")<0) and (img_src.lower().find(".htm")<0) and (img_src.lower().find(".thmx")<0) and (img_src.lower().find(".js")<0) and (img_src.lower().find(".pdf")<0) and (img_src.lower().find(".zip")<0) and (img_src.lower().find(".asp")<0) and (img_src.lower().find(".txt")<0) and (img_src.lower().find(".ico")<0) and (img_src.lower().find(".config")<0) and (img_src.lower().find(".db")<0) and (img_src.lower().find(".psd")<0) and (img_src.lower().find(".doc")<0) and (img_src.lower().find(".p7s")<0) and (img_src.lower().find(".mts")<0) and (img_src.lower().find(".mov")<0):
        download_process_box_full_id_box_file(client, id, str(id_box_file), folder, file_name, cursor_update, image_hash,cursor_aitb,cursor_aitb_x,cursor_update_aitb,cursor_update_aitb_x)
        if(id_tbl_webpage_images is not None):
            if(not id_tbl_webpage_images>0):
                #logic repeats below - making a function seemed to make this more complicated
                #No match found in the table tbl_box_images
                wi_id=0            
                cursor_wi_id = mariadb_connection.cursor(buffered=True, dictionary=True)
                img_path_match_sql = img_path_match.replace("'","''")
                #print(img_path_match_sql)
                cursor_wi_id.execute("SELECT id as wi_id FROM webpage_images.tbl_webpage_images WHERE img_path_match='"+img_path_match_sql+"';")
                #print(cursor_inner)
                #print("rows "+str(cursor_wi_id.rowcount))
                for wi_ids in cursor_wi_id:
                    wi_id=wi_ids["wi_id"]
                    print("id "+str(wi_id))
                    if(wi_id>0):
                        update_image = ("UPDATE tbl_box_images SET `id_tbl_webpage_images` = %s WHERE id=%s;")
                        #print(update_image)
                        cursor_update.execute(update_image, (wi_id, id))
                        mariadb_connection.commit()
                cursor_wi_id.close()
        else:
            #logic repeats from above
            wi_id=0            
            cursor_wi_id = mariadb_connection.cursor(buffered=True, dictionary=True)
            #cursor_inner.execute(select_match, (img_path_match))
            
            img_path_match_sql = img_path_match.replace("'","''")
            #print(img_path_match_sql)
            cursor_wi_id.execute("SELECT id as wi_id FROM webpage_images.tbl_webpage_images WHERE img_path_match='"+img_path_match_sql+"';")
            #print("rows "+str(cursor_wi_id.rowcount))
            for wi_ids in cursor_wi_id:
                wi_id=wi_ids["wi_id"]
                print("id "+str(wi_id))
                if(wi_id>0):
                    update_image = ("UPDATE tbl_box_images SET `id_tbl_webpage_images` = %s WHERE id=%s;")
                    #print(update_image)
                    cursor_update.execute(update_image, (wi_id, id))
                    mariadb_connection.commit()
            cursor_wi_id.close()
            
cursor.close()
cursor_update.close()
cursor_inner.close()
cursor_aitb.close()
cursor_aitb_x.close()
cursor_update_aitb.close()
cursor_update_aitb_x.close()


mariadb_connection.close()    
#/Altars/Altar 10110_files/image009.jpg
#/Altars/Altar 10110_files/image009.jpg

In [31]:
mariadb_connection.close()    

In [ ]:
mariadb_connection.close()   